## [Module Import]

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import scipy.stats as ss
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import scipy.stats as stat
import warnings
warnings.filterwarnings(action='ignore')

## [Functions]

In [2]:
def d1(s,k,r,q,sigma,tau):
    return (np.log(s/k) + (r-q+(sigma**2)/2)*tau) / (sigma*np.sqrt(tau))

In [3]:
def d2(s,k,r,q,sigma,tau):
    return d1(s,k,r,q,sigma,tau) - sigma*np.sqrt(tau)

In [4]:
def delta(s,k,r,q,sigma,tau):
    return ss.norm.cdf(d1(s,k,r,q,sigma,tau))

In [5]:
def modified_delta(s,k,r,q,sigma,tau,delta_u, w_up, w_down):
    
    current_call_price = calloptionPrice(s,k,r,q,sigma,tau)
    
    if 0<= delta_u < 1:
        up_call_price = calloptionPrice(s+s*delta_u,k,r,q,sigma,tau)
        down_call_price = calloptionPrice(s-s*delta_u,k,r,q,sigma,tau)
        
    elif delta_u >= 1:
        up_call_price = calloptionPrice(s+s,k,r,q,sigma,tau)
        down_call_price = calloptionPrice(s-s,k,r,q,sigma,tau)
        
    else:
        up_call_price = 0
        down_call_price = 0
        print("wrong delta")
    
    
    up_delta = (up_call_price - current_call_price)/(s*delta_u)
    down_delta =(current_call_price - down_call_price)/(s*delta_u)
    
    return w_up*up_delta + w_down*down_delta

In [6]:
def gamma(S, K, r, T, sigma):
    d1 = (np.log(S/K) + (r + 0.5 * (sigma ** 2)) * T) / (sigma * np.sqrt(T))    
    return stat.norm.pdf(d1) / (sigma * S * np.sqrt(T))

In [7]:
def calloptionPrice(s,k,r,q,sigma,tau):
    return s*np.exp(-q*tau)*ss.norm.cdf(d1(s,k,r,q,sigma,tau)) - k*np.exp(-r*tau)*ss.norm.cdf(d2(s,k,r,q,sigma,tau))

## [Dynamic Hedging 함수]

In [8]:
"""
### Main Parameter Discription ###

1. simul_or_real: 
    -simul : Simulated Path에 대한 각 Rebalancing 전략별 Dynamic Hedge Performance 측정
    -real : 실제 주식 Path에 대한 각 Rebalancing 전략별 Dynamic Hedge Performance 측정

2. long_or_short:
    -long : Call Option long position에 대한 Dynamic Hedging
    -short: Call Option short position에 대한 Dynamic Hedging
    
3. delta_strategy(세부내용 Report 참조)
    -BSM
    -Modified_max_gamma_ratio
    -Momentum
    -Trigger
    -Sigma_Trigger
    -Sigma_Trigger_delta

4. vol_type:
    -constant : 직전 6개월 Vol. 고정
    -historical : 직전 6개월 Vol. rolling by 1 month
    -IV : 직전 6개월 Implied Vol. 고정 (15% Theoretical Edge 반영)
"""

'\n### Main Parameter Discription ###\n\n1. simul_or_real: \n    -simul : Simulated Path에 대한 각 Rebalancing 전략별 Dynamic Hedge Performance 측정\n    -real : 실제 주식 Path에 대한 각 Rebalancing 전략별 Dynamic Hedge Performance 측정\n\n2. long_or_short:\n    -long : Call Option long position에 대한 Dynamic Hedging\n    -short: Call Option short position에 대한 Dynamic Hedging\n    \n3. delta_strategy(세부내용 Report 참조)\n    -BSM\n    -Modified_max_gamma_ratio\n    -Momentum\n    -Trigger\n    -Sigma_Trigger\n    -Sigma_Trigger_delta\n\n4. vol_type:\n    -constant : 직전 6개월 Vol. 고정\n    -historical : 직전 6개월 Vol. rolling by 1 month\n    -IV : 직전 6개월 Implied Vol. 고정 (15% Theoretical Edge 반영)\n'

In [9]:
def DynamicHedging(simul_or_real,long_or_short, delta_strategy, vol_type ,s0, mu, k, r, q, sigma, T, 
                              numOfCall, timeBtwHedgeRebal, sim_num, stock_name="s", atm_threshold_ratio = 0.1):
    
    global vol_dict
    
    if long_or_short != 'long' and long_or_short != 'short':
        print('input parameter "long_or_short" must be "long" or "short"!')
        return
    
    np.random.seed(1233)

    
    # Simulation 인지 Real 주가 Path 인지에 따라 변수 조정
    if simul_or_real == "simul":
        IV = sigma
        if long_or_short == 'long':
            IV = IV - IV * 0.15
        elif long_or_short == 'short':
            IV = IV + IV * 0.15
        
        T = 26
        timeBtwHedgeRebal = 0.2 # 일일 시뮬레이션
        daysToYears = np.linspace(0,T,int(T/timeBtwHedgeRebal)+1)/52
        total_steps = len(daysToYears)
        dt = timeBtwHedgeRebal/52
        
        
        s = np.zeros((total_steps, sim_num))
        s[0,:] = s0
        
        vol_path = np.zeros((total_steps, sim_num)) + sigma
        
        
        for i in range(total_steps-1):
            z = np.random.randn(sim_num)
            s[i+1,:] = s[i,:] * np.exp((mu-0.5*sigma**2)*dt + np.sqrt(dt)*(sigma*z))
        
        true_call_p = calloptionPrice(s0,k,r,q,sigma,1/2)
        implied_call_p = calloptionPrice(s0,k,r,q,IV,1/2)
        theoretical_edge = abs(true_call_p - implied_call_p)
    
    
    elif simul_or_real == "real":
        
        ### day ###
        daysToYears = np.linspace(0,T,int(T/timeBtwHedgeRebal)+1)/250

        ### stock path ###
        total_steps = len(daysToYears)
        dt = timeBtwHedgeRebal/250
    
        sim_num = 1
        data = pd.read_csv("./data/"+stock_name+"_RV.csv")
        
        if vol_type == "constant":
            vol_path = data.constant_vol.values.reshape(-1,1)
            vol_path = np.insert(vol_path, 0, vol_path[0]).reshape(-1,1)
            
            IV = vol_path[0][0]
            if long_or_short == 'long':
                IV = IV - IV * 0.15
    
            elif long_or_short == 'short':
                IV = IV + IV * 0.15
            
        elif vol_type == "historical":
            vol_path = data.historical_vol_6m.values.reshape(-1,1)
            vol_path = np.insert(vol_path, 0, vol_path[0]).reshape(-1,1)
            
            IV = vol_path[0][0]
            if long_or_short == 'long':
                IV = IV - IV * 0.15
    
            elif long_or_short == 'short':
                IV = IV + IV * 0.15
        
        elif vol_type == "IV":
            vol_path = data.constant_vol.values.reshape(-1,1)
            vol_path = np.insert(vol_path, 0, vol_path[0]).reshape(-1,1)
            
            IV = vol_path[0][0]
            
            if long_or_short == 'long':
                IV = IV - IV * 0.15
                vol_path = np.zeros((total_steps, sim_num)) + IV
            elif long_or_short == 'short':
                IV = IV + IV * 0.15
                vol_path = np.zeros((total_steps, sim_num)) + IV
                

        
        s = pd.read_csv("./data/"+stock_name+".csv").close_y.values.reshape(-1,1)
        s = np.insert(s, 0, s[0]).reshape(-1,1)
        
        k= s[0][0]*1.1
        
        true_call_p = calloptionPrice(s[0][0],k,r,q,vol_dict[stock_name][0],1/2)
        implied_call_p = calloptionPrice(s[0][0],k,r,q,IV,1/2)
        theoretical_edge = abs(true_call_p - implied_call_p)
        
    else:
        print("input simul or real")
        return
    
    
    
    
    
    ### Delta Path 설정: 각각의 Delta Path가 곧 하나의 전략 ###
    
    delta_u_path = np.zeros((total_steps, sim_num))
    up_buffer = np.zeros((total_steps, sim_num))
    down_buffer = np.zeros((total_steps, sim_num))
    gamma_path = np.zeros((total_steps, sim_num))
    max_gamma_path = np.zeros((total_steps, sim_num))
    
    ## BSM : 기존 BSM Delta##
    if delta_strategy == "BSM":
        
        BSM_delta_path = np.zeros((total_steps, sim_num))
        for j in range(sim_num):
            for i in range(total_steps):
                if long_or_short == 'long':
                    BSM_delta_path[i,j] = delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)]) 
                elif long_or_short == 'short':
                    BSM_delta_path[i,j] = -delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)])
        
        delta_path = BSM_delta_path
    
    
    
    ## Modified_max_gamma_ratio: Modified Delta의 delta_u를 max_gamma 대비 Ratio로 계산 ##
    elif delta_strategy == "Modified_max_gamma_ratio":
        
        Modified_delta_path = np.zeros((total_steps, sim_num))
        gamma_path = np.zeros((total_steps, sim_num))
        max_gamma_path = np.zeros((total_steps, sim_num))
        
        for j in range(sim_num):
            for i in range(total_steps):
                if long_or_short == 'long':
                    gamma_path[i,j] = gamma(s[i,j], k, r, daysToYears[len(daysToYears) - (i + 1)],  vol_path[i,j])
                    max_gamma_path[i,j] = gamma(k, k, r, daysToYears[len(daysToYears) - (i + 1)],  vol_path[i,j]) 
                elif long_or_short == 'short':
                    gamma_path[i,j] = -gamma(s[i,j], k, r, daysToYears[len(daysToYears) - (i + 1)],  vol_path[i,j])
                    max_gamma_path[i,j] = -gamma(k, k, r, daysToYears[len(daysToYears) - (i + 1)],  vol_path[i,j])
                
                if i == (total_steps-1):
                    delta_u_path[i,j] = delta_u_path[i-1,j]
                else:
                    delta_u_path[i,j] = (max_gamma_path[i,j]/gamma_path[i,j]) /1000
                    
                    #제한두는 값이 중요하다!
                    if delta_u_path[i,j] > 0.3:
                        delta_u_path[i,j] = 0.3
                    else: 
                        pass
                    
                
                if i == 0:
                    if long_or_short == 'long':
                        Modified_delta_path[i,j] = modified_delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)],
                                                                 delta_u_path[i,j], 0.5, 0.5) 
                    elif long_or_short == 'short':
                        Modified_delta_path[i,j] = -modified_delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)],
                                                                  delta_u_path[i,j], 0.5, 0.5)
                    
                    up_buffer[i,j] = s[i,j] + s[i,j]*delta_u_path[i,j]
                    down_buffer[i,j] = s[i,j] - s[i,j]*delta_u_path[i,j]
                    
                else:
                    #만기시점 가까이에서 한번 delta_u가 튀어버리면 hedge 계속안하게될 가능성있음 추후 조정 필요
                    if down_buffer[i,j] < s[i,j] < up_buffer[i,j]:
                        Modified_delta_path[i,j] = Modified_delta_path[i-1,j]
                        down_buffer[i,j] = down_buffer[i-1,j]
                        up_buffer[i,j] = up_buffer[i-1,j]
                        
                        
                    else:
                        if long_or_short == 'long':
                            Modified_delta_path[i,j] = modified_delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)],
                                                                 delta_u_path[i,j], 0.5, 0.5) 
                        elif long_or_short == 'short':
                            Modified_delta_path[i,j] = -modified_delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)],
                                                                      delta_u_path[i,j], 0.5, 0.5)
                        
                        up_buffer[i,j] = s[i,j] + s[i,j]*delta_u_path[i,j]
                        down_buffer[i,j] = s[i,j] - s[i,j]*delta_u_path[i,j]
                                                      
                            
        delta_path = Modified_delta_path
    
    
    
    ## Momentum : 위의 "Modified_max_gamma_ratio" 전략을 이어받아 momentum에 따라 w1과 w2의 가중치를 조절 ##
    elif delta_strategy == "Momentum":
        
        Momentum_delta_path = np.zeros((total_steps, sim_num))
        gamma_path = np.zeros((total_steps, sim_num))
        max_gamma_path = np.zeros((total_steps, sim_num))
        
        for j in range(sim_num):
            w_up = 0.5
            w_down = 0.5
            for i in range(total_steps):
                if long_or_short == 'long':
                    gamma_path[i,j] = gamma(s[i,j], k, r, daysToYears[len(daysToYears) - (i + 1)],  vol_path[i,j])
                    max_gamma_path[i,j] = gamma(k, k, r, daysToYears[len(daysToYears) - (i + 1)],  vol_path[i,j]) 
                elif long_or_short == 'short':
                    gamma_path[i,j] = -gamma(s[i,j], k, r, daysToYears[len(daysToYears) - (i + 1)],  vol_path[i,j])
                    max_gamma_path[i,j] = -gamma(k, k, r, daysToYears[len(daysToYears) - (i + 1)],  vol_path[i,j])
                
                if i == (total_steps-1):
                    delta_u_path[i,j] = delta_u_path[i-1,j]
                else:
                    delta_u_path[i,j] = (max_gamma_path[i,j]/gamma_path[i,j]) /10
                    
                    #제한두는 값이 중요하다!
                    if delta_u_path[i,j] > 0.3:
                        delta_u_path[i,j] = 0.3
                    else: 
                        pass             
                
                if i < 2:
                    if long_or_short == 'long':
                        Momentum_delta_path[i,j] = modified_delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)],
                                                                 delta_u_path[i,j], w_up, w_down) 
                    elif long_or_short == 'short':
                        Momentum_delta_path[i,j] = -modified_delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)],
                                                                  delta_u_path[i,j], w_up, w_down)
                else:
                    if (s[i,j] > s[i-1,j]) & (s[i-1,j] > s[i-2,j]):
                        w_up = 0.7
                        w_down = 0.3

                    elif (s[i,j] < s[i-1,j]) & (s[i-1,j] < s[i-2,j]):
                        w_up = 0.3
                        w_down = 0.7

                    else:
                        w_up = 0.5
                        w_down = 0.5

                    if long_or_short == 'long':
                        Momentum_delta_path[i,j] = modified_delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)],
                                                                 delta_u_path[i,j], w_up, w_down) 
                    elif long_or_short == 'short':
                        Momentum_delta_path[i,j] = -modified_delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)],
                                                                  delta_u_path[i,j], w_up, w_down)

        delta_path = Momentum_delta_path
        
    
    
    
    ## Trigger: Derman Lecture note에 소개되는 Delta Trigger Strategy 참고 ##
    elif delta_strategy == "Trigger":

        trig =0.02
        Trigger_delta_path = np.zeros((total_steps, sim_num))
        for j in range(sim_num):
            for i in range(total_steps):
                if long_or_short == 'long':
                    if i == 0:
                        Trigger_delta_path[i,j] = delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)])
                    else:
                        Trigger_delta_path[i,j] = delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)])
                        if abs(Trigger_delta_path[i,j] / Trigger_delta_path[i-1,j] - 1) <= trig:
                            Trigger_delta_path[i,j] = Trigger_delta_path[i-1,j]


                elif long_or_short == 'short':
                    if i == 0:
                        Trigger_delta_path[i,j] = -delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)])
                    else:
                        Trigger_delta_path[i,j] = -delta(s[i,j], k, r, q,  vol_path[i,j], daysToYears[len(daysToYears) - (i + 1)])
                        if abs(Trigger_delta_path[i,j] / Trigger_delta_path[i-1,j] - 1) <= trig:
                            Trigger_delta_path[i,j] = Trigger_delta_path[i-1,j]

        delta_path = Trigger_delta_path
    
    
    
    
    ## Sigma_Trigger: Realized Volatility를 Trigger로 둔 전략 ##
    elif delta_strategy == "Sigma_Trigger":
        
        ## sigma trigger ##
        ST = vol_path[0][0]
        trig = 0.02
        ST_delta_path = np.zeros((total_steps, sim_num))
        for j in range(sim_num):
            for i in range(total_steps):
                if long_or_short == 'long':
                    if i == 0:
                        ST_delta_path[i,j] = delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                    elif i < 382:
                        ST_delta_path[i,j] = delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        if abs(ST_delta_path[i,j] / ST_delta_path[i-1,j] - 1) <= trig:
                            ST_delta_path[i,j] = ST_delta_path[i-1,j] 
                    else:
                        ST_delta_path[i,j] = ST_delta_path[i-1,j]
                        if (abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) & (i < 382*2):
                            ST_delta_path[i,j] = delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        elif ((abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*2,j]))*np.sqrt(1/(382*2*dt)) >= ST)) & (i < 382*3):
                            ST_delta_path[i,j] = delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        elif ((abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*2,j]))*np.sqrt(1/(382*2*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*3,j]))*np.sqrt(1/(382*3*dt)) >= ST)):
                            ST_delta_path[i,j] = delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])  



                elif long_or_short == 'short':
                    if i == 0:
                        ST_delta_path[i,j] = -delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])           
                    elif i < 382:
                        ST_delta_path[i,j] = -delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        if abs(ST_delta_path[i,j] / ST_delta_path[i-1,j]-1) <= trig:
                            ST_delta_path[i,j] = ST_delta_path[i-1,j]
                    else:
                        ST_delta_path[i,j] = ST_delta_path[i-1,j]
                        if (abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) & (i < 382*2):
                            ST_delta_path[i,j] = -delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        elif ((abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*2,j]))*np.sqrt(1/(382*2*dt)) >= ST)) & (i < 382*3):
                            ST_delta_path[i,j] = -delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        elif ((abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*2,j]))*np.sqrt(1/(382*2*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*3,j]))*np.sqrt(1/(382*3*dt)) >= ST)):
                            ST_delta_path[i,j] = -delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])                          

        
        delta_path = ST_delta_path 
        
    
    ## Sigma_Trigger_delta: 위의 "Sigma_Trigger" 전략을 만기에 가까워 질때 Delta Trigger로 전환 ##
    elif delta_strategy == "Sigma_Trigger_delta":
        ST = vol_path[0][0]   ## sigma trigger ##
        trig = 0.02
        ST2_delta_path = np.zeros((total_steps, sim_num))
        for j in range(sim_num):
            for i in range(total_steps):
                if long_or_short == 'long':
                    if i == 0:
                        ST2_delta_path[i,j] = delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                    elif (i < 382) | ((s[i,j]>=k) & (i >= (total_steps)*0.9)):
                        ST2_delta_path[i,j] = delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        if abs(ST2_delta_path[i,j] / ST2_delta_path[i-1,j]-1) <= trig:
                            ST2_delta_path[i,j] = ST2_delta_path[i-1,j]                          
                    else:
                        ST2_delta_path[i,j] = ST2_delta_path[i-1,j]
                        if (abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) & (i < 382*2):
                            ST2_delta_path[i,j] = delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        elif ((abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*2,j]))*np.sqrt(1/(382*2*dt)) >= ST)) & (i < 382*3):
                            ST2_delta_path[i,j] = delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        elif ((abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*2,j]))*np.sqrt(1/(382*2*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*3,j]))*np.sqrt(1/(382*3*dt)) >= ST)):
                            ST2_delta_path[i,j] = delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])  

                elif long_or_short == 'short':
                    if i == 0:
                        ST2_delta_path[i,j] = -delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                    elif (i < 382) | ((s[i,j]>=k) & (i >= (total_steps)*0.9)):
                        ST2_delta_path[i,j] = -delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        if abs(ST2_delta_path[i,j] / ST2_delta_path[i-1,j]-1) <= trig:
                            ST2_delta_path[i,j] = ST2_delta_path[i-1,j]                            
                    else:
                        ST2_delta_path[i,j] = ST2_delta_path[i-1,j]
                        if (abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) & (i < 382*2):
                            ST2_delta_path[i,j] = -delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        elif ((abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*2,j]))*np.sqrt(1/(382*2*dt)) >= ST)) & (i < 382*3):
                            ST2_delta_path[i,j] = -delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])
                        elif ((abs(np.log(s[i,j]/s[i-382,j]))*np.sqrt(1/(382*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*2,j]))*np.sqrt(1/(382*2*dt)) >= ST) | (abs(np.log(s[i,j]/s[i-382*3,j]))*np.sqrt(1/(382*3*dt)) >= ST)):
                            ST2_delta_path[i,j] = -delta(s[i,j], k, r, q, vol_path[i,j], daysToYears[len(daysToYears)-(i+1)])     
        
        delta_path = ST2_delta_path  
    

    else:
        print("#### Missing Strategy of Delta Path #####")
        
        
    #Terminal Delta 값을 Moneyness에 따라 조정 
    if long_or_short == 'long':
        for j in range(sim_num):
            if s[-1,j] > k:
                delta_path[-1,j] = 1
            else:
                delta_path[-1,j] = 0
                
    elif long_or_short == 'short':
        for j in range(sim_num):
            if s[-1,j] > k:
                delta_path[-1,j] = -1
            else:
                delta_path[-1,j] = 0
    
    
    ### Shares purchase ###
    sharesPurchased = np.zeros((total_steps, sim_num))
        
    for j in range(sim_num):
        for i in range(total_steps):
            if i == 0:
                sharesPurchased[i,j] = -delta_path[i,j] * numOfCall
            else:
                sharesPurchased[i,j] = -(delta_path[i,j] - delta_path[i-1,j]) * numOfCall

    
    
    # 거래비용 고려
    
    # 여기에다가 sharesPurchased가 (-)일 때, 그만큼 주식을 팔아야 하니깐, 거래비용 추가부분 (*0.001) 해주어야한다.
    ### Cost of shares purchased ###
    
    # 거래비용 고려한 코드
    costOfSharesPurchased = s * sharesPurchased * (np.ones((total_steps, sim_num)) + (sharesPurchased < 0) * (-0.001))
    transaction_cost = s * sharesPurchased * (sharesPurchased < 0) * (-0.001)

    
    
    
    # 현금을 갖고 있다고 가정하고(또는 현금을 차입했다고 생각하고), 갖고 있는 현금(또는 빌린 현금)에 대한 이자율까지 고려
    # 이 부분에 기존 현금을 갖고 있고, 거기에 대해 헷징 비용이 부과되며, 전체적으로 이자율이 고려되는 코드를 짜야함.
    
    ### Interest Cost & Cumulative cost including interest ###
    # long : 현금을 쓰고 시작 (cost가 (+)에서 시작), short : 현금을 들고 시작 (cost가 (-)에서 시작)
    
    if long_or_short == 'long':
        initial_cost = (calloptionPrice(s0, k, r, q, IV, T/52)) * numOfCall
    else:
        initial_cost = (-1) * (calloptionPrice(s0, k, r, q, IV, T/52)) * numOfCall
                                 
    
    interestCost = np.zeros((total_steps, sim_num))
    cumCostInclInterest = np.zeros((total_steps, sim_num))
    
    for j in range(sim_num):
        for i in range(total_steps):
            if i == 0:
                
                # 기존 : 그냥 헷징 코스트만 구하는 것.
                #cumCostInclInterest[i,j] = costOfSharesPurchased[i,j]
                
                # 현재 : 주어진 현금에 헷징 코스트를 차감 하는 구조.
                cumCostInclInterest[i,j] = initial_cost + costOfSharesPurchased[i,j]
                interestCost[i,j] = cumCostInclInterest[i,j] * r * dt

            else:
                cumCostInclInterest[i,j] = cumCostInclInterest[i-1,j] + costOfSharesPurchased[i,j] + interestCost[i-1,j]

                if i == total_steps-1:
                    interestCost[i,j] = 0.0
                else:
                    interestCost[i,j] = cumCostInclInterest[i,j] * r * dt
                
    # 기존 : hedging cost를 구하는 것이었음.            
    #hedgingCost = cumCostInclInterest[-1,:] - k * numOfCall * (s[-1, :] > k)
    
    #현재 : 만기 시점에 남은 돈을 구하는 것 -> 옵션이 행사되면 long(cost 감소), short(cost 증가)
    if long_or_short == 'long':
        final_money = cumCostInclInterest[-1,:] + k * numOfCall * (s[-1, :] > k)
    else:
        final_money = cumCostInclInterest[-1,:] - k * numOfCall * (s[-1, :] > k)
    

    
    
    
    """
        ATM 정의를 어떻게 할 것인가?
        1. 행사가격 += 일정비율? : ATM에서 행사 된 것, 안된 것 다 포함되어서 hedging ratio가 낮게 나옴.
        2. 행사가격 - 일정비율? : ATM은 "행사가 안되었다"고 생각할 수 있기 때문에 적절할듯?
        그 외 다른방법?
    """

    final_prices = s[-1,:]
    atm_lower_bound = (1-atm_threshold_ratio) * k
    atm_higher_bound = k
    ITM_list = final_money[final_prices >= atm_higher_bound]
    ATM_list = final_money[(final_prices < atm_higher_bound)&(final_prices >= atm_lower_bound)]
    OTM_list = final_money[final_prices < atm_lower_bound]
    
    
    hedging_performance = final_money.std() / (calloptionPrice(s0,k,r,q,IV,T/52) * numOfCall)
    ITM_hedging_performance = ITM_list.std() / (calloptionPrice(s0,k,r,q,IV,T/52) * numOfCall)
    ATM_hedging_performance = ATM_list.std() / (calloptionPrice(s0,k,r,q,IV,T/52) * numOfCall)
    OTM_hedging_performance = OTM_list.std() / (calloptionPrice(s0,k,r,q,IV,T/52) * numOfCall)
    
    
    
    moneyness_res = [ITM_list, ATM_list, OTM_list, ITM_hedging_performance, ATM_hedging_performance, OTM_hedging_performance]
    
    
    final_money = final_money.mean()
    if simul_or_real == 'simul':
        transaction_cost = transaction_cost.sum() / sim_num
    elif simul_or_real == 'real':
        transaction_cost = transaction_cost.sum()
        

    return s, delta_path, gamma_path, max_gamma_path, delta_u_path, up_buffer, down_buffer, final_money, hedging_performance, theoretical_edge, cumCostInclInterest, transaction_cost, moneyness_res

### [Final Result 추출 함수 1] : Simulation 주가 Path

In [10]:
def make_final_result_simulation(strategy,file_name):
    performance_table = pd.DataFrame(index=["Total",'ITM','ATM','OTM'], columns=strategy)
    final_money_table = pd.DataFrame(index=["Total",'ITM','ATM','OTM'], columns=strategy)

    for ds in strategy:
        s, delta_path, gamma_path, max_gamma_path, delta_u_path, up_buffer, down_buffer, final_money, hedging_performance, theoretical_edge, cumCostInclInterest, transaction_cost, moneyness_res = DynamicHedging('simul','long', ds, 'IV' ,s0, mu, k, r, q, sigma, T, numOfCall, timeBtwHedgeRebal, sim_num, stock_name="s", atm_threshold_ratio = 0.1)

        final_money_table.loc["Total",ds] = final_money.mean()
        final_money_table.loc["ITM",ds] = moneyness_res[0].mean()
        final_money_table.loc["ATM",ds] = moneyness_res[1].mean()
        final_money_table.loc["OTM",ds] = moneyness_res[2].mean()
        
        performance_table.loc["Total",ds] = hedging_performance
        performance_table.loc["ITM",ds] = moneyness_res[3]
        performance_table.loc["ATM",ds] = moneyness_res[4]
        performance_table.loc["OTM",ds] = moneyness_res[5]
        

    with pd.ExcelWriter(file_name+'.xlsx') as writer:  
        performance_table.to_excel(writer, sheet_name='performance_table')
        final_money_table.to_excel(writer, sheet_name='final_money_table')
        
    return performance_table, final_money_table

### [Final Result 추출 함수 2] : Real 주가 Path

In [11]:
# Real 주가 Path 사용시 Result.xlsx 추출 함수
def make_final_result_real(stock_names,long_short, vol_types, strategy, file_name):
    global s0 , mu, k, r, q, sigma, T, numOfCall, timeBtwHedgeRebal
    
    
    result = {}
    for name in stock_names:
        print(name)
        for pos in long_short:
            data = pd.DataFrame(columns=strategy, index=vol_types)
            for vol in vol_types:
                for st in strategy:
                    s, delta_path, gamma_path, max_gamma_path, delta_u_path, up_buffer, down_buffer, final_money, hedging_performance, theoretical_edge, cumCostInclInterest, transaction_cost, moneyness_res = DynamicHedging("real",pos,st,vol, s0, mu, k, r, q, sigma, T, numOfCall, timeBtwHedgeRebal, sim_num, name)
                    
                    data.loc[vol, st] = (int(final_money),int(transaction_cost), len(np.unique(delta_path)))
                    result[(name, pos)] = data
    
    
    with pd.ExcelWriter(file_name+'.xlsx') as writer:  
        for k in result.keys():
            result[k].to_excel(writer, sheet_name=(str(k[0]) +"_"+ str(k[1])))
    
    
    return result

In [12]:
# 간단한 Result 추출 함수
def make_result(simul_or_real,long_or_short, delta_strategy, vol_type, sim_num, stock_name="s", path_num = 0):
    global s0 , mu, k, r, q, sigma, T, numOfCall, timeBtwHedgeRebal
    
    s, delta_path, gamma_path, max_gamma_path, delta_u_path, up_buffer, down_buffer, final_money, hedging_performance, theoretical_edge, cumCostInclInterest, transaction_cost, moneyness_res = DynamicHedging(simul_or_real,long_or_short,delta_strategy,vol_type, s0, mu, k, r, q, sigma, T, numOfCall, timeBtwHedgeRebal, sim_num, stock_name)
    
    num= path_num
    data = pd.DataFrame(np.vstack([s[:,num], delta_path[:,num],delta_u_path[:,num],down_buffer[:,num],up_buffer[:,num]
                                   ,gamma_path[:,num],max_gamma_path[:,num], cumCostInclInterest[:,num]]), 
                        index=["s", "delta_path", "delta_u_path", "up_buffer", "down_buffer","gamma_path", "max_gamma_path","cumCost"] ).T.iloc[:,:]
    
    print("final_money: ", final_money)
    print("hedging_performance: ",hedging_performance)
    print("theoretical_edge: ",theoretical_edge)
    print("transaction_cost: ",transaction_cost)
    
    
    return data, np.mean(final_money), delta_path

## [TermProject P #1] Simulation - Long call(34%), Short call(46%)

Pre_term 에서는 Short call 포지션을 헷징하였음. 이제는, Long&Short call 합성 포지션을 헷징해야함. 조건은 아래와 같음
- S = 50000
- RV = 40%(IV : Long(34%), Short(46%). 즉, 40 +(-)40 * 15% = 40 +(-)6% = 46, 34%)
- r = 2% (= 0.02)
- q = 0%
- K = 55000
- Option type = European Vanilla Call
- the number of contract : 100,000 contracts

#### 고정된 값(문제에서 조건으로 준 값)

In [13]:
# 문제 조건 (fixed)
k = 55000
r = 0.02
q = 0
sigma = 0.4
s0 = 50000
numOfCall = 1
T = 125 # 단위 : 일 (데이터 범위 총 125일 사용)

#### 고정되지 않은 값(문제에서 주지 않아서 개발자가 임의로 정한 값)

In [14]:
# 우리가 정하는 값(modifiable)
long_or_short = 'long'
atm_threshold_ratio = 0.1
mu = 0.13 # 기존 pre-term 값을 사용

# 1분에 1번 델타헷징. ##timeBtwHedgeRebal = 1 이면 하루한번 hedge!
timeBtwHedgeRebal = 1/382 #ex>하루를 382분으로 나눔  ##0.2면 1주를 1/5 한것 이므로 하루 한번 헤지! # 단위 : 주, (5, 4, 2, 1, 0.5, 0.25)
# timeBtwHedgeRebal = 0.0026387434554974 # 엄밀 126(일)/47750(# of DataPoint) # 사실 위의 값과 거의 같다.

# 시뮬레이션은 1000번만 한다고 가정.
sim_num = 1000

## Simulation Examples

### ex1) Simul_BSM

In [15]:
res1 = make_result("simul", "long", "BSM", "historical", 1000)

final_money:  -702.7970283342645
hedging_performance:  0.15283359165907762
theoretical_edge:  830.0044320540255
transaction_cost:  120.76925595550924


### ex2) Simul_Modified_max_gamma_ratio

In [16]:
res2 = make_result("simul", "long", "Modified_max_gamma_ratio", "historical", 1000)

final_money:  -708.4486657534184
hedging_performance:  0.19772455777338258
theoretical_edge:  830.0044320540255
transaction_cost:  128.24191103783545


### ex3) Simul_Momentum

In [17]:
res3 = make_result("simul", "long", "Momentum", "historical", 1000)

final_money:  -619.1137425867039
hedging_performance:  0.41834539908939156
theoretical_edge:  830.0044320540255
transaction_cost:  183.8041444396123


### ex4) Simul_Trigger

In [18]:
res4 = make_result("simul", "long", "Trigger", "historical", 1000)

final_money:  -710.8066273683141
hedging_performance:  0.15510115658656123
theoretical_edge:  830.0044320540255
transaction_cost:  117.42837902784598


### ex5) Simul_Sigma_Trigger

In [19]:
res5 = make_result("simul", "long", "Sigma_Trigger", "historical", 1000)

final_money:  -710.8066273683141
hedging_performance:  0.15510115658656123
theoretical_edge:  830.0044320540255
transaction_cost:  117.42837902784598


### ex6) Simul_Sigma_Trigger_delta

In [20]:
res6 = make_result("simul", "long", "Sigma_Trigger_delta", "historical", 1000)

final_money:  -710.8066273683141
hedging_performance:  0.15510115658656123
theoretical_edge:  830.0044320540255
transaction_cost:  117.42837902784598


## Final_Simulation_Result (Excel 저장, 첨부파일 참고) 

In [21]:
strategy_list = ['Sigma_Trigger','Sigma_Trigger_delta','BSM','Modified_max_gamma_ratio','Momentum','Trigger']

In [22]:
final_sim_result_to_excel = make_final_result_simulation(strategy_list, "final_simulation")

## [TermProject P #2] 실제 주가 데이터를 이용한 분석

1. 기준일 직전 6개월 SMA 변동성이 30% 내외인 종목 수개(최소 4개). 다른 변동성도
무방함. (과거 변동성이 실현 변동성과 같지 않으며, 한 개의 주가 path에 의존)
2. 10% OTM E_V_Call을 일정한 margi을 확보한 상태에서 10만개 기준으로 매수/
매도. Margin은 변동성에 비례하여 15% 가감함.
예) 30% 변동성이라면(30*15%=4.5%), 매수는 25.5%, 매도는 34.5%의
내재변동성으로 옵션 프리미엄을 산출할 것.
3. 거래비용을 고려한다(매도 시 매도대금*0.1%를 감안). 다만, bid-ask sprea는
무시함. 자금비용은 년 2%로 해서 일별 계산함.

In [23]:
# 각 주가 Path의 기본적인 Vol_Data
# Vol_dict = {주식명: Front_vol, Back_vol, Total_vol}
vol_dict = {'삼성전자': [0.2854981077672668, 0.2338300978239685, 0.2623155442023024],
            '두산중공업': [0.5003216640357473, 0.7620044849744695, 0.6531566725759548],
            '셀트리온헬스케어': [0.45873605461247124, 0.33358926941422684, 0.41434169971342094],
            '롯데칠성': [0.32814852106807696, 0.2619066354598787, 0.2974295409932259],
            '네이버': [0.35778236001178315, 0.33265906182906496, 0.3468251229159798],
            '현대건설': [0.4538914857882747, 0.32779936502946194, 0.3980698908801952],
            'SK네트웍스': [0.39337880616575827, 0.26371918402377176, 0.3332501241995625], 
            '코미팜':[0.6730826919816825,0.48231776889519784,0.5882105761860837], 
            '제주항공':[0.5543889494163243,0.32912320908722864,0.452382627041269]
           }

## 실제 주가 Path Examples

### ex1) Real_BSM

In [24]:
res1 = make_result("real", "long", "BSM", "historical", 1, "삼성전자")

final_money:  4532.234092821105
hedging_performance:  0.0
theoretical_edge:  533.4062841374252
transaction_cost:  2442.5160044374848


### ex2) Real_Modified_max_gamma_ratio

In [25]:
res2 = make_result("real", "long", "Modified_max_gamma_ratio", "historical", 1, "삼성전자")

final_money:  5588.222927866911
hedging_performance:  0.0
theoretical_edge:  533.4062841374252
transaction_cost:  2850.121974674525


### ex3) Real_Momentum

In [26]:
res3 = make_result("real", "long", "Momentum", "historical", 1, "삼성전자")

final_money:  6599.8726313693915
hedging_performance:  0.0
theoretical_edge:  533.4062841374252
transaction_cost:  8439.362172414445


### ex4) Real_Trigger

In [27]:
res4 = make_result("real", "long", "Trigger", "historical", 1, "삼성전자")

final_money:  5494.750987879583
hedging_performance:  0.0
theoretical_edge:  533.4062841374252
transaction_cost:  349.8478056404407


### ex5) Real_Sigma_Trigger

In [28]:
res5 = make_result("real", "long", "Sigma_Trigger", "historical", 1, "삼성전자")

final_money:  5172.222011438062
hedging_performance:  0.0
theoretical_edge:  533.4062841374252
transaction_cost:  1173.2541017636681


### ex6) Real_Sigma_Trigger_delta

In [29]:
res6 = make_result("real", "long", "Sigma_Trigger_delta", "historical", 1, "삼성전자")

final_money:  5196.039193848759
hedging_performance:  0.0
theoretical_edge:  533.4062841374252
transaction_cost:  1160.4210630820023


## Final_Real_Result (Excel 저장)  (1시간 가량 소요 , 첨부파일 참고)

In [30]:
stock_names = ['삼성전자', '네이버', '두산중공업', '롯데칠성', 'SK네트웍스', '셀트리온헬스케어', '현대건설', '코미팜', '제주항공']

In [31]:
long_short = ["long", "short"]

In [32]:
vol_types = ["historical" , "IV"]

In [33]:
strategy = ["BSM", "Modified_max_gamma_ratio", "Momentum", "Trigger", "Sigma_Trigger", "Sigma_Trigger_delta"]

In [34]:
final_result_real = make_final_result_real(stock_names, long_short, vol_types, strategy, "final_result_real")

삼성전자
네이버
두산중공업
롯데칠성
SK네트웍스
셀트리온헬스케어
현대건설
코미팜
제주항공
